In [ ]:
import mplfinance as mlf 
import pandas as pd
import matplotlib.pyplot as plt
#import drawing_function.function as function
from joblib import Parallel,delayed
import copy
mlf.__version__

### 绘图的jupyter版本，便于实验

#### MACD绘制
为了让MACD直方图有颜色，必须使用另一个函数进行获取一个颜色列表，输入进入Mplfinance进行绘图

In [ ]:
def get_macd_colors(data):
    colors=[]
    for i in range(len(data)):
        if data[i]<0:
            colors.append("green")  
        elif data[i]>0:
            colors.append("red") 
        else:
            colors.append("#F0FFFF")
    return colors

In [ ]:
def subdrawing(data,s,profit):
    labelDate=data.index[0]
    code=data.iloc[0,0]
    labelv=profit.query('(labelDate == @labelDate)')
    data=data.iloc[1:,:]
    drawing_list=[
        mlf.make_addplot(data['MA5'],type='line',color='b',panel=0),
        mlf.make_addplot(data['MA10'],type='line',color='r',panel=0),
        mlf.make_addplot(data['MA15'],type='line',color='g',panel=0),
        mlf.make_addplot(data['MA20'],type='line',color='y',panel=0),
        mlf.make_addplot(data['MACD'],type='line',panel=2,color='b'),
        mlf.make_addplot(data['Signal'],type='line',panel=2,color='y'),
        mlf.make_addplot(data['Histogram'],type='bar',panel=2,color=get_macd_colors(data['Histogram']))
    ]
    fig,axes=mlf.plot(
        data,type='candle',
        volume=True,ylabel="",
        style=s,addplot=drawing_list,
        figscale=3,returnfig=True,panel_ratios=(5,3,2)
    )
    axes[0].yaxis.set_ticks([])
    axes[2].yaxis.set_ticks([])
    axes[2].set_ylabel("")
    axes[4].yaxis.set_ticks([])
    axes[5].xaxis.set_ticks([])
    axes[5].yaxis.set_ticks([])
    for i in range(6):
        axes[i].spines[['top','bottom','left','right']].set_visible(False)
    try:
        labelv=float(labelv.iloc[0,1])
    except:
        #print(labelDate,code)
        plt.close() 
    else:
        path=""
        if labelv>0:
            path="/home/hydu/CNN/image/2/"
            name=f"2-{code}-{labelDate}.png"
            path=path+name
        elif labelv==0:
            path="/home/hydu/CNN/image/1/"
            name=f"1-{code}-{labelDate}.png"
            path=path+name
        else:
            path="/home/hydu/CNN/image/0/"
            name=f"0-{code}-{labelDate}.png"
            path=path+name
        plt.savefig(fname=path,dpi=20)  
        plt.close()     

#### 真正的绘图函数
首先计算MACD各项指标，随后使用mlf的主题制作函数制作初级主题设置，进行第一步的图主题绘画。<br>
随后使用mlf接口获得matplotlib形式的mlf返回对象，首先去除标签、名字等。随后根据matplotlib对象图，直接去除spine。

In [ ]:
def drawing(this_data,profit20,code,style):
    #this_data=copy.copy(data.query('code == @code'))
    MA5=this_data['Close'].rolling(window=5).mean()
    MA10=this_data['Close'].rolling(window=10).mean()
    MA15=this_data['Close'].rolling(window=15).mean()
    MA20=this_data['Close'].rolling(window=20).mean()
    Ema12=this_data['Close'].ewm(span=12,adjust=False).mean()
    Ema26=this_data['Close'].ewm(span=26,adjust=False).mean()
    MACD=Ema12-Ema26
    Signal=MACD.ewm(span=9,adjust=False).mean()
    Histogram=MACD-Signal
    Histogram.columns=['Macd']
    this_data['Signal']=Signal
    this_data['MACD']=MACD
    this_data['Histogram']=Histogram
    this_data['MA5']=MA5
    this_data['MA10']=MA10
    this_data['MA15']=MA15
    this_data['MA20']=MA20
    row,line=this_data.shape
    for i in range(0,row-21):
        if i<=20:  #如果时间不到20天，平均线不存在
            continue
        slice_data=this_data.iloc[i:i+21,:]
        #print(slice_data)
        subdrawing(data=slice_data,s=style,profit=profit20)
    
    #mlf.show()

#### 基本数据处理
得到了h5文件（文件内部由不同group组成，同时每一组中包含不同的dataset），必须使用特殊函数，同时指定读取数据(key='data)。此外数据索引必须是DateTime格式，同时需要重新命名。

In [ ]:
path="/data/AI_Project/DataSets/AdjDaily/"
filenames=[]
for i in range(2007,2008):
    filename=f"{path}{i}.h5"
    filenames.append(filename)
data=pd.read_hdf("/data/AI_Project/DataSets/AdjDaily/2006.h5",key='data')
for filename in filenames:
    tempdata=pd.read_hdf(filename,key='data')
    data=pd.concat([data,tempdata])
profit20=pd.read_hdf("/data/AI_Project/Labels/Ret20.h5")
stock_code=sorted(list(data.loc[:,'code'].unique()),key=str.lower)

data.index=pd.to_datetime(data.index,format="%Y%m%d")
profit20.index=pd.to_datetime(profit20.index,format="%Y%m%d")
profit20['labelDate']=pd.to_datetime(profit20["labelDate"],format="%Y%m%d")
data.columns=columns=['code', 'preclose', 'Open', 'High', 'Low', 'Close', 'Volume', 'amount','trdnum']

mc=mlf.make_marketcolors(up='r',down='g',wick={'up':'blue','down':'orange'},volume={'up':'r','down':'g'})
s=mlf.make_mpf_style(base_mpf_style='yahoo',marketcolors=mc,mavcolors=['r','g','b','orange'],gridstyle='')

In [ ]:
print(len(stock_code))

In [ ]:
data.head()

In [ ]:
profit20.head()

#### 数据切分
将数据每20天切分一次，进行绘图，同时应该注意如何标注标签。

In [ ]:
data_list=[]
profit_list=[]
for code in stock_code:
    temp_data=copy.copy(data.query('code == @code'))
    temp_profit=copy.copy(profit20.query('code == @code'))
    data_list.append(temp_data)
    profit_list.append(temp_profit)

In [ ]:
Parallel(n_jobs=64)(delayed(drawing)(data,profit,style=s,code=code) for data,profit,code in zip(data_list,profit_list,stock_code))